In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip "/content/drive/MyDrive/deep_learning_project/rare_species 1.zip" -d "/content/"

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ConvNeXtBase
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split

In [ ]:
import os
import pandas as pd

base_dir_drive = "/content/drive/MyDrive/deep_learning_project"
image_dir = "/content/rare_species 1"

df = pd.read_csv(os.path.join(base_dir_drive, "metadata.csv"))
df["file_path"] = df["file_path"].str.replace("\\", "/", regex=False)
df["file_path"] = df["file_path"].apply(lambda x: os.path.join(image_dir, x))

In [ ]:
X = df["file_path"]  # Caminhos das imagens
y = df["family"]  # Classes correspondentes

In [ ]:
n_classes = len(y.unique())
n_classes

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

train_data = pd.DataFrame({"file_path": X_train, "family": y_train})
test_data = pd.DataFrame({"file_path": X_test, "family": y_test})

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_data["label"] = label_encoder.fit_transform(train_data["family"])
test_data["label"] = label_encoder.transform(test_data["family"])  # usar o mesmo encoder
n_classes = len(label_encoder.classes_)

In [ ]:
import tensorflow as tf

def load_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0  # normalização
    return image, label

In [ ]:
import tensorflow as tf

# rotation = tf.keras.layers.RandomRotation(0.1)
# zoom = tf.keras.layers.RandomZoom(0.1)

rotation = tf.keras.layers.RandomRotation(0.2)
zoom = tf.keras.layers.RandomZoom(0.2)

def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    image = rotation(image)
    image = zoom(image)
    return image, label

In [ ]:
BATCH_SIZE_HEAD = 32
BATCH_SIZE_FINE = 64
#BATCH_SIZE_TESTAR = 96
AUTOTUNE = tf.data.AUTOTUNE

# 1. Pipeline base único (sem batch ainda)
base_train_ds = tf.data.Dataset.from_tensor_slices((train_data["file_path"].values, train_data["label"].values))
base_train_ds = base_train_ds.map(load_image, num_parallel_calls=AUTOTUNE)
base_train_ds = base_train_ds.map(augment, num_parallel_calls=AUTOTUNE)
base_train_ds = base_train_ds.shuffle(buffer_size=1000).prefetch(AUTOTUNE)

base_val_ds = tf.data.Dataset.from_tensor_slices((test_data["file_path"].values, test_data["label"].values))
base_val_ds = base_val_ds.map(load_image, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

# 2. Versões batched
train_ds = base_train_ds.batch(BATCH_SIZE_HEAD)
val_ds = base_val_ds.batch(BATCH_SIZE_HEAD)

train_ds_ft = base_train_ds.batch(BATCH_SIZE_FINE)
val_ds_ft = base_val_ds.batch(BATCH_SIZE_FINE)

# train_ds_testar = base_train_ds.batch(BATCH_SIZE_TESTAR)
# val_ds_testar = base_val_ds.batch(BATCH_SIZE_TESTAR)

In [ ]:
checkpoint_dir = os.path.join(base_dir_drive, "modelos")

callbacks_finetune_model2 = [
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1),
    EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True, verbose=1),
    ModelCheckpoint(
        filepath=os.path.join(checkpoint_dir, "best_model2_effnetv2_finetune.keras"),
        save_best_only=True,
        monitor="val_loss",
        verbose=1,
    ),
]

callbacks_head_model2 = [
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1),
    EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True, verbose=1),
    ModelCheckpoint(
        filepath=os.path.join(checkpoint_dir, "best_model2_effnetv2_head.keras"),
        save_best_only=True,
        monitor="val_loss",
        verbose=1,
    ),
]

In [ ]:
from tensorflow.keras.applications import ConvNeXtBase
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, BatchNormalization, Input
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.regularizers import l2

# 1. Modelo base
base_model2 = ConvNeXtBase(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)
base_model2.trainable = False  # congelado no início

# 2. Construir modelo com head agressivamente regularizada
inputs = Input(shape=(224, 224, 3))
x = base_model2(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dense(1024, activation="gelu", kernel_regularizer=l2(0.005))(x)
x = Dropout(0.6)(x)
x = Dense(512, activation="gelu", kernel_regularizer=l2(0.005))(x)
x = Dropout(0.6)(x)
output = Dense(n_classes, activation="softmax")(x)

model2 = Model(inputs=inputs, outputs=output)

# 3. Compilar (AdamW + regularização agressiva)
model2.compile(
    optimizer=AdamW(learning_rate=1e-4, weight_decay=3e-4),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

# 4. Treinar head com callbacks já definidos
history_model2_head = model2.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=callbacks_head_model2  # já definidos
)

# 5. Descongelar últimas 200 camadas para fine-tuning
for layer in base_model2.layers[-300:]:
    layer.trainable = True

# 6. Recompilar para fine-tuning
model2.compile(
    optimizer=AdamW(learning_rate=5e-6, weight_decay=3e-4),
    loss=SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

# 7. Treinar com fine-tuning
history_model2_finetuned = model2.fit(
    train_ds_ft,
    validation_data=val_ds_ft,
    epochs=55,
    callbacks=callbacks_finetune_model2
)